In [ ]:
import pandas as pd
import numpy as np
import time
import lightgbm as lgb
import sys

In [2]:
t1 = time.time()
df_train = pd.read_csv('/home/kai/data/kaggle/talkingdata/wl/data/features/train_mean_combine5_0405.csv')
# df_test = pd.read_csv('/home/kai/data/kaggle/talkingdata/wl/data/features/test_fold_last_in_12_count.csv')
t2 = time.time()
print('training loading done! Time: {}'.format(t2 - t1))
print('size is: {}'.format(sys.getsizeof(df_train) / 1024 ** 3))
print('length is {}'.format(len(df_train)))

training loading done! Time: 406.17874217033386
size is: 17.60319436341524
length is 10272438


# Specify Categorical Columns

In [3]:
# remove minute and second
categorical_col = [ 'device', 'app', 'os', 'channel', 'hour',  ]
target = 'is_attributed'
feature_cols = list(set(df_train.columns) - set([target]) - set(['minute', 'second', 'ip', 'intesthh' , 'timestamp']))

# Get Validation

In [4]:
ratio = 0.2
val_length = int(len(df_train) * ratio)
trainset = df_train.iloc[:val_length]
valset = df_train.iloc[val_length:]
print(sys.getsizeof(trainset)/ 1024 **3)
print(len(trainset))

3.520637921988964
2054487


In [5]:
# from sklearn.model_selection import train_test_split
# trainset, valset = train_test_split(df_train,test_size=0.2, random_state=42)
# print(sys.getsizeof(trainset)/ 1024 **3)
# print(len(trainset))

In [6]:
y_train = trainset[target].values
y_val = valset[target].values

lgb_train = lgb.Dataset(trainset[feature_cols], y_train, categorical_feature = categorical_col)
lgb_val = lgb.Dataset(valset[feature_cols], y_val, categorical_feature = categorical_col)

zeros = len(y_train[y_train == 0])
scale_pos_weight = len(y_train[y_train == 0]) / len(y_train) * 100
import gc
# del df_train
gc.collect()

print(scale_pos_weight)

99.74801495458478


# Train Lightgbm

In [7]:
params = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'num_rounds': 2000,
        'learning_rate': 0.1,
        'num_leaves': 13,
        'num_threads': 8, # best speed: set to number of real cpu cores, which is vCPU/2
        'device': 'cpu',
        'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
        'min_data_in_leaf': 200,  #minimal number of data in one leaf. Can be used to deal with over-fitting
        'feature_fraction': 0.7, #For example, if set to 0.8, will select 80% features before training each tree.  speed up training / deal with over-fitting
        'feature_fraction_seed': 1,
        'early_stopping_round':50,
        'bagging_fraction': 0.9, #Randomly select part of data without resampling
        'bagging_freq': 1, #frequency for bagging, 0 means disable bagging. k means will perform bagging at every k iteration. to enable bagging, bagging_fraction should be set as well
        'bagging_seed': 1,
        #'max_bin': 255,
        'verbose': 0,
        'scale_pos_weight': scale_pos_weight,
        'metric' : ['binary_logloss', 'auc']
    }


model = lgb.train(params, train_set=lgb_train, valid_sets=lgb_val, verbose_eval=10)

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:668: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 50 rounds.
[10]	valid_0's auc: 0.96867	valid_0's binary_logloss: 0.238569
[20]	valid_0's auc: 0.97084	valid_0's binary_logloss: 0.119272
[30]	valid_0's auc: 0.972153	valid_0's binary_logloss: 0.0786964
[40]	valid_0's auc: 0.973147	valid_0's binary_logloss: 0.0632258
[50]	valid_0's auc: 0.974024	valid_0's binary_logloss: 0.0570363
[60]	valid_0's auc: 0.973525	valid_0's binary_logloss: 0.0539315
[70]	valid_0's auc: 0.973412	valid_0's binary_logloss: 0.0524903
[80]	valid_0's auc: 0.973304	valid_0's binary_logloss: 0.0516355
[90]	valid_0's auc: 0.973169	valid_0's binary_logloss: 0.0499404
Early stopping, best iteration is:
[46]	valid_0's auc: 0.974064	valid_0's binary_logloss: 0.0586817


In [8]:
from sklearn.metrics import roc_auc_score
pred_val = model.predict(valset[feature_cols])
print(roc_auc_score(y_val,pred_val))

0.974063906745


In [9]:
importance = pd.Series(model.feature_importance(), index=feature_cols)
importance = importance.sort_values(ascending=False)
if len(model.feature_importance()) != len(feature_cols):
    raise ValueError('Feature importance has length: {}, \n while feature number is {}'.
                     format(len(model.feature_importance()), len(feature_cols)))
importance.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/output/importance_train_mean_all_0405_combine5.csv')

In [10]:
importance

channel                                  95
ip_mean                                  55
os                                       36
app_os_channel_mean                      29
app                                      25
ip_app_os_day_hour_mean                  21
app_device_os_channel_mean               15
app_os_channel_day_mean                  14
app_os_channel_intesthh_mean             12
app_device_os_channel_intesthh_mean      12
ip_app_mean                              12
ip_app_device_channel_mean                9
app_device_os_day_mean                    9
ip_device_channel_mean                    8
ip_app_device_mean                        8
ip_app_intesthh_mean                      7
ip_channel_mean                           7
ip_device_hour_intesthh_mean              7
ip_os_channel_day_hour_mean               7
app_channel_intesthh_mean                 6
ip_app_channel_day_intesthh_mean          5
app_os_channel_day_intesthh_mean          5
app_device_os_channel_day_mean  

In [25]:
aa = model.predict(valset[feature_cols])

In [12]:
df_test = pd.read_csv('/home/kai/data/kaggle/talkingdata/wl/data/features/test_fold_last_in_12_mean.csv')


In [13]:
df_test = df_test[list(valset[feature_cols].columns)]
bb = model.predict(df_test)

In [14]:
list(set(valset[feature_cols].columns) - set(df_test.columns))

[]

In [15]:
bb

array([ 0.00102839,  0.00131542,  0.00099425, ...,  0.00100567,
        0.00130841,  0.00099425])

In [16]:
list(valset[feature_cols].columns)

['app_day_minute_mean',
 'app',
 'second_mean',
 'device_os_hour_mean',
 'os_day_second_mean',
 'device_day_hour_mean',
 'day',
 'ip_hour_minute_mean',
 'channel_day_hour_mean',
 'ip_device_hour_mean',
 'channel_hour_mean',
 'os_day_mean',
 'minute_second_mean',
 'app_device_os_mean',
 'app_day_hour_mean',
 'ip_channel_mean',
 'app_hour_mean',
 'app_device_day_mean',
 'app_day_second_mean',
 'os_mean',
 'os_channel_mean',
 'app_channel_minute_mean',
 'device_channel_minute_mean',
 'os_minute_mean',
 'device_os_channel_mean',
 'day_minute_second_mean',
 'channel_mean',
 'channel_hour_minute_mean',
 'ip_channel_day_mean',
 'os_channel_day_mean',
 'device_hour_minute_mean',
 'ip_device_os_mean',
 'device_channel_second_mean',
 'os_channel_second_mean',
 'device_day_minute_mean',
 'hour_minute_second_mean',
 'device',
 'app_day_mean',
 'os_day_minute_mean',
 'os_hour_second_mean',
 'second',
 'device_mean',
 'app_hour_second_mean',
 'app_channel_day_mean',
 'day_hour_second_mean',
 'channe

In [18]:
df_sub.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/submission/train_fold_last_in_12csv.gz', compression='gzip', index=False)

In [17]:
# prediction
df_test_raw = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/test.csv')

df_sub = pd.DataFrame()
df_sub['click_id'] = df_test_raw['click_id']
df_sub['is_attributed'] = bb

In [18]:
df_sub.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/submission/train_fold_6_in_6_last_mean_0403_3_noscale.csv.gz', compression='gzip', index=False)

In [46]:
valset.head(1)

,ip,app,device,os,channel,day,hour,timestamp,minute,second,...,ip_app_device_count,ip_app_os_count,ip_app_channel_count,ip_device_os_count,ip_device_channel_count,ip_os_channel_count,app_device_os_count,app_device_channel_count,app_os_channel_count,device_os_channel_count
21638813,2348,15,1,13,245,8,23,1510182584,9,44,...,608.0,190.0,222.0,1963.0,313.0,96.0,2829042.0,4553892.0,1062250.0,1579048.0


In [45]:
df_test.head(1)

,ip,app,device,os,channel,day,hour,timestamp,minute,second,...,ip_app_device_count,ip_app_os_count,ip_app_channel_count,ip_device_os_count,ip_device_channel_count,ip_os_channel_count,app_device_os_count,app_device_channel_count,app_os_channel_count,device_os_channel_count
0,5744,9,1,3,107,10,4,1510286400,0,0,...,64.0,1.0,10.0,2.0,34.0,0.0,195701.0,504264.0,11217.0,174692.0
